# 4.4.1 Fitting Logistic Regression Models

Logistic regression models fit by maximum likelihood, the log-likelihood for N observations is (4.19):
$$
l(\theta)=\sum_{i=1}^N \log p_{g_i}(x_i, \theta),
$$

We discuss in detail the two-class case, it is convenient to code the two-class $g_i$ via 0/1 response $y_i$. The log-likelihood can be written (4.20):
$$
\begin{align}
l(\beta) &= \sum_{i=1}^N \left \{y_i \log p(x_i; \beta) + (1-y_i)log(1 - p(x_i;\beta)) \right \}\\
&= \sum_{i=1}^N \left \{ 
  y_i(\beta^Tx_i - log(1+e^{\beta^Tx_i})) -
  (1-y_i)log(1+e^{\beta^Tx_i})
\right\}\\
&= \sum_{i=1}^N \left \{ y_i\beta^Tx_i - \log (1+e^{\beta^Tx_i}) \right\}
\end{align}
$$

Here $\beta = \{\beta_{10}, \beta_1\}$, and we assume that the vector of inputs $x_i$ includes the constant term 1.

To maximize the log-likelihood, we set its derivative to zero (4.21):
$$
\begin{align}
\cfrac{\partial l(\beta)}{\partial \beta} &=\cfrac{\partial \left [
\sum_{i=1}^N \left \{ y_i\beta^Tx_i - \log (1+e^{\beta^Tx_i}) \right\}
\right]}
{\partial \beta}\\
&= \sum_{i=1}^N \left \{ y_ix_i - \cfrac{e^{\beta^Tx_i}}{1+e^{\beta^Tx_i}}   x_i\right\}\\
&= \sum_{i=1}^N x_i(y_i-p(x_i; \beta))
\end{align}
$$

which are p + 1 equations *nonlinear* in $\beta$. Notice that since the first component of $x_i$ is 1, the first score equation specifies that $\sum_{i=1}^N y_i = \sum_{i=1}^N p(x_i;\beta)$, the *expected* number of class ones matches the observed number (ane hence also class twos).